In [1]:
# editable parameters:
quesname='q1/'
logfilename='a1.log'

In [2]:
# Initialisation

#q1 of ass1

import numpy as np
import math
import torch as t
import torch 
import os


# import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

import matplotlib.pyplot as plt
%matplotlib inline



import logging
log_dir = '../../log'
os.makedirs(log_dir, exist_ok=True)

log_dir=os.path.join(log_dir, quesname)

os.makedirs(log_dir, exist_ok=True)

logfile=os.path.join(log_dir, logfilename)
logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



In [3]:

# part 1 and 2:
# dataset loading:
training_data = datasets.FashionMNIST(
    root="../../data/",
    train=True,
    download=True,
    transform=ToTensor() # Normalisation step
    
)

# note that ToTensor not just converts the image into a tensor but also normalises its intensity in range 0 to 1


test_data = datasets.FashionMNIST(
    root="../../data/",
    train=False,
    download=True,
    transform=ToTensor() # Normalisation step
)

# Create data loaders for training and testing

batchSize=64
train_loader = DataLoader(training_data, batch_size=batchSize, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batchSize, shuffle=False)

In [4]:
# part 3:

import torch.nn as nn

# builtin classifier:

# Define the model

# model saving
# Define a directory to save models
save_dir = '../../saved_models/q1/'
os.makedirs(save_dir, exist_ok=True)

model = nn.Sequential(

    # we need to flatten the input data from 2d tensor into 1d tensor of 784 values (28*28) since data is in form of images
    nn.Flatten(),  # Flatten the input data

    nn.Linear(in_features=784, out_features=10),  # Linear layer   
    # nn.Linear(in_features=784, out_features=batchSize),  # Linear layer   

    nn.BatchNorm1d(10),
    # this normalsises input tensors by scaling and shifting
    # makes mean 0 and variance 1

    # softmax function, or normalized exponential function converts a vector of K real numbers into a probability distribution of K possible outcomes
    nn.Softmax(dim=1)  # Softmax activation

    
)

# Specify the path to the saved checkpoint file
# model_path = '../../saved_models/model_checkpoint.pth'

# tk
model_path = os.path.join(save_dir, 'ignore.pth')

# model_path = os.path.join(save_dir, 'model_checkpoint.pth')

# Check if the file exists before loading
if os.path.exists(model_path):
    # Load the model checkpoint
    checkpoint = torch.load(model_path)

    # Load the model's state_dict
    model.load_state_dict(checkpoint)
    print("Model loaded successfully.")
else:
    print(f"Checkpoint file '{model_path}' not found. Model not loaded.")

# custom made classifier
'''
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()

        # Define the linear layer
        # we used 28*28 input and 10 labels because we are working with MNIST database
        self.fc = nn.Linear(28 * 28, 10)  # 28*28 input features, 10 output classes
        
        # Define an activation function (e.g., softmax for classification)

        # softmax function, or normalized exponential function converts a vector of K real numbers into a probability distribution of K possible outcomes
        self.softmax = nn.Softmax(dim=1)

    #IMPORTANT the name 'forward' is not arbitrary; it's a convention in PyTorch, and this method is automatically called when we pass data through our model.
    def forward(self, x):

        # we need to flatten the input data from 2d tensor into 1d tensor of 784 values (28*28) since data is in form of images
        x = x.view(x.size(0), -1)  # Flatten the input tensor

        x = self.fc(x)  # Apply the linear transformation
        x = self.softmax(x) # Apply the activation function
        return x
'''
print('model created')

Checkpoint file '../../saved_models/q1/ignore.pth' not found. Model not loaded.
model created


In [5]:
print("Model structure before training: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure before training:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Softmax(dim=1)
) 


Layer: 1.weight | Size: torch.Size([10, 784]) | Values : tensor([[ 0.0243, -0.0251,  0.0104,  ..., -0.0245,  0.0044,  0.0202],
        [-0.0075,  0.0140,  0.0048,  ..., -0.0297,  0.0040, -0.0074]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([10]) | Values : tensor([-0.0257,  0.0339], grad_fn=<SliceBackward0>) 

Layer: 2.weight | Size: torch.Size([10]) | Values : tensor([1., 1.], grad_fn=<SliceBackward0>) 

Layer: 2.bias | Size: torch.Size([10]) | Values : tensor([0., 0.], grad_fn=<SliceBackward0>) 



In [6]:
# part 4:

# Define the loss function
criterion = nn.CrossEntropyLoss()

import torch.optim as optim
# Define the optimizer (e.g., SGD)

learning_rate=0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

'''
about SGD: Stochastic Gradient Descent:
basically it performs seqeuntial updation of paramters after each epoch, using formula :
parameter = parameter - learning_rate * gradient

'''

print('part 4 done')



part 4 done


In [11]:
# part 5
# Define the number of training epochs
num_epochs = 20

import time
t1=time.time()
# Training loop
for epoch in range(num_epochs):
    # Set the model in training mode
    model.train()
    
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in train_loader:  # Replace train_loader with your data loader
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()
        
        # Track the running loss
        running_loss += loss.item()
    
    # Calculate and print the average loss for this epoch
    average_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {average_loss:.4f}")
    # dasdsd

print("Training complete")
t2=time.time()
training_time=t2-t1
print('time taken to train : ',training_time)



# Save the model checkpoint to the specified directory
# model_path = os.path.join(save_dir, 'model_checkpoint.pth')

# tk
# torch.save(model.state_dict(), model_path)


Epoch [1/20] - Loss: 1.6760
Epoch [2/20] - Loss: 1.6720
Epoch [3/20] - Loss: 1.6692
Epoch [4/20] - Loss: 1.6666
Epoch [5/20] - Loss: 1.6636
Epoch [6/20] - Loss: 1.6621
Epoch [7/20] - Loss: 1.6592
Epoch [8/20] - Loss: 1.6576
Epoch [9/20] - Loss: 1.6560
Epoch [10/20] - Loss: 1.6547
Epoch [11/20] - Loss: 1.6534
Epoch [12/20] - Loss: 1.6517
Epoch [13/20] - Loss: 1.6505
Epoch [14/20] - Loss: 1.6491
Epoch [15/20] - Loss: 1.6492
Epoch [16/20] - Loss: 1.6461
Epoch [17/20] - Loss: 1.6467
Epoch [18/20] - Loss: 1.6444
Epoch [19/20] - Loss: 1.6431
Epoch [20/20] - Loss: 1.6437
Training complete
time taken to train :  32.416686058044434


In [8]:
print("Model structure after training: ", model, "\n\n")
print("jere")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure after training:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Softmax(dim=1)
) 


jere
Layer: 1.weight | Size: torch.Size([10, 784]) | Values : tensor([[ 0.0243, -0.0251,  0.0105,  ..., -0.0275,  0.0041,  0.0202],
        [-0.0075,  0.0140,  0.0048,  ..., -0.0315,  0.0037, -0.0075]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([10]) | Values : tensor([-0.0257,  0.0339], grad_fn=<SliceBackward0>) 

Layer: 2.weight | Size: torch.Size([10]) | Values : tensor([1.2631, 1.3120], grad_fn=<SliceBackward0>) 

Layer: 2.bias | Size: torch.Size([10]) | Values : tensor([0.0091, 0.0129], grad_fn=<SliceBackward0>) 



In [9]:
# part 6 :
# testing 

# Set the model to evaluation mode
model.eval()

correct_predictions = 0
total_predictions = 0

# Iterate over the testing dataset

k=0
for inputs, labels in test_loader:  # Use your test data loader
    k+=1
    # Forward pass to obtain predictions
    outputs = model(inputs)
    
    # Get the predicted class for each example in the batch
    _, predicted = torch.max(outputs, dim=1)
    
    # Count correct predictions in this batch
    batch_correct_predictions = (predicted == labels).sum().item()
    correct_predictions += batch_correct_predictions
    
    # Count total predictions in this batch

    batch_total_predictions = labels.size(0)
    total_predictions += batch_total_predictions 

    # Print the running accuracy for this batch
    # Update running accuracy
    batch_accuracy=batch_correct_predictions/batch_total_predictions * 100.0
    print(f"Accuracy on this batch {batch_accuracy:.2f}%")

    accuracy = (correct_predictions / total_predictions) * 100.0
    print(f"Accuracy till now:  testing dataset: {accuracy:.2f}%")

print()
print("number of tests:",k)

# Calculate the accuracy
accuracy = (correct_predictions / total_predictions) * 100.0
print(f"Accuracy on the testing dataset: {accuracy:.2f}%")
print('time taken to train : ',training_time)


# saving accuracy


def loginfo():        


    # Configure logging
    # logging.basicConfig(filename='../../log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Your accuracy calculation
    # accuracy = 0.85  # Replace this with your actual accuracy value

    # Log the accuracy

    # moved these to top cell
    '''
    log_dir = '../../log'
    os.makedirs(log_dir, exist_ok=True)
    log_dir=os.path.join(log_dir, 'q1/')
    
    os.makedirs(log_dir, exist_ok=True)
    logfile=os.path.join(log_dir, 'a1.log')
    logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    '''

    logging.info(f"model :  linear,  batchnorm, softmax")
    logging.info(f"Accuracy: {accuracy:.2f}%")
    logging.info(f"Training Time: {training_time:.2f} seconds")
    logging.info(f"Number of epochs: {num_epochs:.2f} ")
    logging.info(f"Test Session Finished \n")
    

# tk
loginfo()
logging.shutdown()
print("data saved")



Accuracy on this batch 71.88%
Accuracy till now:  testing dataset: 71.88%
Accuracy on this batch 81.25%
Accuracy till now:  testing dataset: 76.56%
Accuracy on this batch 89.06%
Accuracy till now:  testing dataset: 80.73%
Accuracy on this batch 79.69%
Accuracy till now:  testing dataset: 80.47%
Accuracy on this batch 82.81%
Accuracy till now:  testing dataset: 80.94%
Accuracy on this batch 85.94%
Accuracy till now:  testing dataset: 81.77%
Accuracy on this batch 82.81%
Accuracy till now:  testing dataset: 81.92%
Accuracy on this batch 73.44%
Accuracy till now:  testing dataset: 80.86%
Accuracy on this batch 76.56%
Accuracy till now:  testing dataset: 80.38%
Accuracy on this batch 75.00%
Accuracy till now:  testing dataset: 79.84%
Accuracy on this batch 82.81%
Accuracy till now:  testing dataset: 80.11%
Accuracy on this batch 73.44%
Accuracy till now:  testing dataset: 79.56%
Accuracy on this batch 85.94%
Accuracy till now:  testing dataset: 80.05%
Accuracy on this batch 82.81%
Accuracy